# Introduction: Manual Feature Engineering (part two)

In this notebook we will expand on the Introduction to Manual Feature Engineering notebook. We will use the aggregation and value counting functions developed in that notebook in order to incorporate information from the `previous_application`, `POS_CASH_balance`, `installments_payments`, and `credit_card_balance` data files. We already used the information from the `bureau` and `bureau_balance` in the previous notebook and were able to improve our competition score compared to using only the `application` data. 

The definitions of these data files are:

* previous_application: previous applications for loans at Home Credit of clients who have loans in the application data. Each current loan in the application data can have multiple previous loans. Each previous application has one row and is identified by the feature SK_ID_PREV.
* POS_CASH_BALANCE: monthly data about previous point of sale or cash loans clients have had with Home Credit. Each row is one month of a previous point of sale or cash loan, and a single previous loan can have many rows.
* credit_card_balance: monthly data about previous credit cards clients have had with Home Credit. Each row is one month of a credit card balance, and a single credit card can have many rows.
* installments_payment: payment history for previous loans at Home Credit. There is one row for every made payment and one row for every missed payment.

## Functions 

We spent quite a bit of time developing two functions in the previous notebook:

* `agg_numeric`: calculate aggregation statistics for numeric variables.
* `count_cats`: compute counts of each category in a categorical variable and calculate the normalized counts. 

Together, these two functions can extract information about both the numeric and categorical data in a dataframe. Our general approach will be to apply both of these functions to the dataframes, grouping by the client id, `SK_ID_CURR`. For the `POS_CASH_balance`, `credit_card_balance`, and `installment_payments`, we can first group by the `SK_ID_PREV`, the unique id for the previous loan. Then we will group the resulting dataframe by the `SK_ID_CURR` to calculate the aggregation statistics for each client acorss all of their previous loans. If that's a little confusing, we'll walk through what we are doing!

### Function to Aggregate Numeric Data

In [5]:
# pandas and numpy for data manipulation
import pandas as pd
import numpy as np

# matplotlib and seaborn for plotting
import matplotlib.pyplot as plt
import seaborn as sns

# Suppress warnings from pandas
import warnings
warnings.filterwarnings('ignore')

plt.style.use('fivethirtyeight')

In [8]:
def agg_numeric(df, group_var, df_name):
    """Aggregates the numeric values in a dataframe. This can
    be used to create features for each instance of the grouping variable.
    
    Args:
        df (dataframe): the dataframe to calculate the statistics on
        group_var (string): the variable by which to group df
        df_name (string): the variable used to rename the columns
        
    Returns:
        agg (dataframe): a dataframe with the statistics aggregated for 
        all numeric columns. Each instance of the grouping variable will have 
        the statistics (mean, min, max, sum; currently supported) calculated. 
        The columns are also renamed to keep track of features created.
    
    """
    
    # First calculate counts
    counts = pd.DataFrame(df.groupby(group_var, as_index = False)[df.columns[2]].count()).rename(columns = {df.columns[2]: '%s_counts' % df_name})
    
    # Group by the specified variable and calculate the statistics
    agg = df.groupby(group_var).agg(['mean', 'max', 'min', 'sum']).reset_index()
    
    # Need to create new column names
    columns = [group_var]
    
    # Iterate through the variables names
    for var in agg.columns.levels[0]:
        # Skip the grouping variable
        if var != group_var:
            # Iterate through the stat names
            for stat in agg.columns.levels[1][:-1]:
                # Make a new column name for the variable and stat
                columns.append('%s_%s_%s' % (df_name, var, stat))
              
    #  Rename the columns
    agg.columns = columns
    
    # Merge with the counts
    agg = agg.merge(counts, on = group_var, how = 'left')
    
    return agg

### Function to Calculate Categorical Counts

In [13]:
def count_cats(df, group_var, df_name):
    """Computes value counts of categories in all categorical variables for a dataframe.
    The value counts are calculated for every instance of the group_var. The dataframe that is
    returned includes the overall counts of each occurence of the grouping variable 
    as well as the value counts of categories for each categorical variable for each 
    occurence of the grouping variable
    
    Args:
        df (dataframe): The dataframe to calculate the value counts for.
        group_var (string): the variable by which to group df
        df_name (string): the variable used to rename the columns
    
    Returns:
        tg (dataframe): a dataframe with value counts of each category in every categorical variable
        of the original df. The overall counts are also merged into the dataframe.
    """
    
    # Select the categorical columns
    cats = df.select_dtypes('object')

    # Make sure to put the identifying id on the column
    cats[group_var] = df[group_var]

    cats_cols = list(cats.columns)

    # Iterate through each categorical column
    for col in cats_cols:
        if col == group_var:
            next
        else:
            # Group by the grouping variable, select the column, calculate value counts
            tg = cats.groupby(group_var)[col].value_counts()

            # Rename the column
            tg = pd.DataFrame(tg).rename(columns = {col: '%s_counts' % col}).reset_index()

            # Pivot table with counts the value to aggregate, index is the grouping variable, 
            # columns are the categorical name, the aggfunc is the mean and fill
            # in missing values with np.nan
            table = pd.pivot_table(tg, values = '%s_counts' % col, index = group_var, 
                                   columns = col, aggfunc = 'mean', fill_value = None)

            # Remove the table name
            table.columns.name = None

            # New names of columns
            table_columns = ['%s_%s_counts' % (col, x) for x in table.columns]
            table.columns = table_columns

            # Merge the created table with the categorical dataframe
            cats = cats.merge(table, left_on = group_var, right_index = True, how = 'left')
            cats = cats.drop_duplicates()
            cats = cats.drop(columns = [col])

    # Get the counts for each client
    counts = pd.DataFrame(df.groupby(group_var)[group_var].count()).rename(columns = {group_var: '%s_counts' % df_name})

    # Merge the counts into the categorical data
    cats = cats.merge(counts, left_on = group_var, right_index = True, how = 'left')
    cats = cats.set_index(group_var)

    # Normalize all the columns by the total_counts
    for col in list(cats.columns):
        cats['%s_normalized' % col] = cats[col] / cats['%s_counts' % df_name]
    
    # Reset the index to make group var a column
    cats = cats.reset_index()
    
    # Drop the duplicate rows
    cats = cats.drop_duplicates()
    
    return cats

### Function for KDE Plots of Variable

We also made a function that plots the distribution of variable colored by the value of `TARGET` (either 1 for did not repay the loan or 0 for did repay the loan). We can use this function to visually examine any new variables we create. This also calculates the correlation cofficient of the variable with the target which can be used as an approximation of whether or not the created variable will be useful. 

In [10]:
# Plots the disribution of a variable colored by value of the target
def kde_target(var_name, df):
    
    # Calculate the correlation coefficient between the new variable and the target
    corr = df['TARGET'].corr(df[var_name])
    
    # Calculate medians for repaid vs not repaid
    avg_repaid = df.ix[df['TARGET'] == 0, var_name].median()
    avg_not_repaid = df.ix[df['TARGET'] == 1, var_name].median()
    
    plt.figure(figsize = (12, 6))
    
    # Plot the distribution for target == 0 and target == 1
    sns.kdeplot(df.ix[df['TARGET'] == 0, var_name], label = 'TARGET == 0')
    sns.kdeplot(df.ix[df['TARGET'] == 1, var_name], label = 'TARGET == 1')
    
    # label the plot
    plt.xlabel(var_name); plt.ylabel('Density'); plt.title('%s Distribution' % var_name)
    plt.legend();
    
    # print out the correlation
    print('The correlation between %s and the TARGET is %0.4f' % (var_name, corr))
    # Print out average values
    print('Median value for loan that was not repaid = %0.4f' % avg_not_repaid)
    print('Median value for loan that was repaid =     %0.4f' % avg_repaid)
    

Let's deal with one dataframe at a time. First up is the `previous_applications`. This has one row for every previous loan a client had at Home Credit. A client can have multiple previous loans which is why we need to aggregate statistics for each client.

In [6]:
previous = pd.read_csv('../input/previous_application.csv')
previous.head()

,SK_ID_PREV,SK_ID_CURR,NAME_CONTRACT_TYPE,AMT_ANNUITY,AMT_APPLICATION,AMT_CREDIT,AMT_DOWN_PAYMENT,AMT_GOODS_PRICE,WEEKDAY_APPR_PROCESS_START,HOUR_APPR_PROCESS_START,...,NAME_SELLER_INDUSTRY,CNT_PAYMENT,NAME_YIELD_GROUP,PRODUCT_COMBINATION,DAYS_FIRST_DRAWING,DAYS_FIRST_DUE,DAYS_LAST_DUE_1ST_VERSION,DAYS_LAST_DUE,DAYS_TERMINATION,NFLAG_INSURED_ON_APPROVAL
0,2030495,271877,Consumer loans,1730.430,17145.0,17145.0,0.0,17145.0,SATURDAY,15,...,Connectivity,12.0,middle,POS mobile with interest,365243.0,-42.0,300.0,-42.0,-37.0,0.0
1,2802425,108129,Cash loans,25188.615,607500.0,679671.0,NaN,607500.0,THURSDAY,11,...,XNA,36.0,low_action,Cash X-Sell: low,365243.0,-134.0,916.0,365243.0,365243.0,1.0
2,2523466,122040,Cash loans,15060.735,112500.0,136444.5,NaN,112500.0,TUESDAY,11,...,XNA,12.0,high,Cash X-Sell: high,365243.0,-271.0,59.0,365243.0,365243.0,1.0
3,2819243,176158,Cash loans,47041.335,450000.0,470790.0,NaN,450000.0,MONDAY,7,...,XNA,12.0,middle,Cash X-Sell: middle,365243.0,-482.0,-152.0,-182.0,-177.0,1.0
4,1784265,202054,Cash loans,31924.395,337500.0,404055.0,NaN,337500.0,THURSDAY,9,...,XNA,24.0,high,Cash Street: high,NaN,NaN,NaN,NaN,NaN,NaN


In [15]:
previous_agg = agg_numeric(previous.drop(columns = ['SK_ID_PREV']), group_var = 'SK_ID_CURR', df_name = 'previous_loans')
previous_agg.head()

,SK_ID_CURR,previous_loans_AMT_ANNUITY_mean,previous_loans_AMT_ANNUITY_max,previous_loans_AMT_ANNUITY_min,previous_loans_AMT_ANNUITY_sum,previous_loans_AMT_APPLICATION_mean,previous_loans_AMT_APPLICATION_max,previous_loans_AMT_APPLICATION_min,previous_loans_AMT_APPLICATION_sum,previous_loans_AMT_CREDIT_mean,...,previous_loans_DAYS_LAST_DUE_sum,previous_loans_DAYS_TERMINATION_mean,previous_loans_DAYS_TERMINATION_max,previous_loans_DAYS_TERMINATION_min,previous_loans_DAYS_TERMINATION_sum,previous_loans_NFLAG_INSURED_ON_APPROVAL_mean,previous_loans_NFLAG_INSURED_ON_APPROVAL_max,previous_loans_NFLAG_INSURED_ON_APPROVAL_min,previous_loans_NFLAG_INSURED_ON_APPROVAL_sum,previous_loans_counts
0,100001,3951.000,3951.000,3951.000,3951.000,24835.50,24835.5,24835.5,24835.5,23787.00,...,-1619.0,-1612.000000,-1612.0,-1612.0,-1612.0,0.000000,0.0,0.0,0.0,1
1,100002,9251.775,9251.775,9251.775,9251.775,179055.00,179055.0,179055.0,179055.0,179055.00,...,-25.0,-17.000000,-17.0,-17.0,-17.0,0.000000,0.0,0.0,0.0,1
2,100003,56553.990,98356.995,6737.310,169661.970,435436.50,900000.0,68809.5,1306309.5,484191.00,...,-3163.0,-1047.333333,-527.0,-1976.0,-3142.0,0.666667,1.0,0.0,2.0,3
3,100004,5357.250,5357.250,5357.250,5357.250,24282.00,24282.0,24282.0,24282.0,20106.00,...,-724.0,-714.000000,-714.0,-714.0,-714.0,0.000000,0.0,0.0,0.0,1
4,100005,4813.200,4813.200,4813.200,4813.200,22308.75,44617.5,0.0,44617.5,20076.75,...,-466.0,-460.000000,-460.0,-460.0,-460.0,0.000000,0.0,0.0,0.0,1


In [16]:
previous_cats = count_cats(previous, group_var = 'SK_ID_CURR', df_name = 'previous_loans')
previous_cats.head()

,SK_ID_CURR,NAME_CONTRACT_TYPE_Cash loans_counts,NAME_CONTRACT_TYPE_Consumer loans_counts,NAME_CONTRACT_TYPE_Revolving loans_counts,NAME_CONTRACT_TYPE_XNA_counts,WEEKDAY_APPR_PROCESS_START_FRIDAY_counts,WEEKDAY_APPR_PROCESS_START_MONDAY_counts,WEEKDAY_APPR_PROCESS_START_SATURDAY_counts,WEEKDAY_APPR_PROCESS_START_SUNDAY_counts,WEEKDAY_APPR_PROCESS_START_THURSDAY_counts,...,PRODUCT_COMBINATION_Cash X-Sell: middle_counts_normalized,PRODUCT_COMBINATION_POS household with interest_counts_normalized,PRODUCT_COMBINATION_POS household without interest_counts_normalized,PRODUCT_COMBINATION_POS industry with interest_counts_normalized,PRODUCT_COMBINATION_POS industry without interest_counts_normalized,PRODUCT_COMBINATION_POS mobile with interest_counts_normalized,PRODUCT_COMBINATION_POS mobile without interest_counts_normalized,PRODUCT_COMBINATION_POS other with interest_counts_normalized,PRODUCT_COMBINATION_POS others without interest_counts_normalized,previous_loans_counts_normalized
0,271877,0,3,0,0,0,0,2,1,0,...,0.000000,0.000000,0.000000,0.666667,0.0,0.333333,0.0,0.0,0.0,1.0
1,108129,2,3,1,0,0,0,0,1,2,...,0.000000,0.166667,0.000000,0.333333,0.0,0.000000,0.0,0.0,0.0,1.0
2,122040,2,1,1,0,1,0,0,0,0,...,0.000000,0.250000,0.000000,0.000000,0.0,0.000000,0.0,0.0,0.0,1.0
3,176158,22,1,0,0,3,8,1,0,1,...,0.043478,0.000000,0.043478,0.000000,0.0,0.000000,0.0,0.0,0.0,1.0
4,202054,18,6,1,0,3,5,2,1,6,...,0.040000,0.080000,0.000000,0.000000,0.0,0.040000,0.0,0.0,0.0,1.0
